# Assignment 3

In this assignment, we will focus on healthcare. This data set is made available by MIT. It contains data about 9,026 heartbeat measurements. Each row represents a single measurement (captured on a timeline). There are a total of 80 data points (columns). This is a multiclass classification task: predict whether the measurement represents a normal heartbeat or other anomalies. 

## Description of Variables

You will use the **hearbeat_cleaned.csv** data set for this assignment. Each row represents a single measurement. Columns labeled as T1 from T80 are the time steps on the timeline (there are 80 time steps, each time step has only one measurement). 

The last column is the target variable. It shows the label (category) of the measurement as follows:<br>
0 = Normal<br>
1 = Supraventricular premature beat<br>
2 = Premature ventricular contraction<br>
3 = Fusion of ventricular and normal beat<br>
4 = Unclassifiable beat

## Goal

Use the data set **hearbeat_cleaned.csv** to predict the column called **Target**. The input variables are columns labeled as **T1 to T80**. 

## Submission:

Please save and submit this Jupyter notebook file. The correctness of the code matters for your grade. **Readability and organization of your code is also important.** You may lose points for submitting unreadable/undecipherable code. Therefore, use markdown cells to create sections, and use comments where necessary.


# Note:

The data is cleaned up. There are no unqueal length sequences. And, there is no zero padding. So, you shouldn't use any `Masking` layer (like I mentioned in the lecture). 

# Read and Prepare the Data (1 points)

In [15]:
# As I was not having tensorflow I tried to install the same
!pip install scikeras[tensorflow]

In [16]:
# Common imports
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd

In [17]:
#We will predict the "Target" value in the data set:

heartbeat = pd.read_csv("heartbeat_cleaned.csv")
heartbeat.head()

,T1,T2,T3,T4,T5,T6,T7,T8,T9,T10,...,T72,T73,T74,T75,T76,T77,T78,T79,T80,Target
0,0.987,0.892,0.461,0.1130,0.1490,0.1900,0.1650,0.1620,0.1470,0.1380,...,0.1970,0.1970,0.1960,0.2030,0.201,0.1990,0.2010,0.205,0.2080,0
1,1.000,0.918,0.621,0.1330,0.1050,0.1250,0.1170,0.0898,0.0703,0.0781,...,0.1950,0.1910,0.1520,0.1720,0.207,0.2110,0.2070,0.207,0.1720,0
2,1.000,0.751,0.143,0.1040,0.0961,0.0519,0.0442,0.0416,0.0364,0.0857,...,0.2260,0.2420,0.2440,0.2860,0.468,0.8160,0.9770,0.452,0.0519,0
3,1.000,0.740,0.235,0.0464,0.0722,0.0567,0.0103,0.0155,0.0284,0.0155,...,0.0851,0.0747,0.0515,0.0593,0.067,0.0361,0.1210,0.451,0.8690,0
4,1.000,0.833,0.309,0.0191,0.1010,0.1200,0.1040,0.0874,0.0765,0.0765,...,0.2050,0.4210,0.8030,0.9510,0.467,0.0000,0.0519,0.082,0.0628,0


In [19]:
heartbeat.shape

(7960, 81)

In [21]:
#I have dropped the target variable since we are trying to predict it
y = heartbeat['Target']
x = heartbeat.drop('Target', axis=1)

In [24]:
#Split data into train and test
from sklearn.model_selection import train_test_split

train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=0.3)

In [26]:
#Check the values of the train_y data set
train_y[30:]

720     0
3822    0
54      0
7833    4
2205    0
       ..
2006    0
4076    0
5384    2
3767    0
2733    0
Name: Target, Length: 5542, dtype: int64

In [27]:
#Convert input variables to a 2-D array with float data type
train_x= np.array(train_x)
test_x= np.array(test_x)

train_x = train_x.astype(np.float32)
test_x = test_x.astype(np.float32)

In [28]:
#Keras expects a different input format:
#Data needs to have 3 dimensions

train_x = np.reshape(train_x, (train_x.shape[0], train_x.shape[1], 1))
test_x = np.reshape(test_x, (test_x.shape[0], test_x.shape[1], 1))



In [29]:
train_x.shape, train_y.shape

((5572, 80, 1), (5572,))

# Find the baseline (0.5 point)

In [31]:
heartbeat['Target'].value_counts()/len(heartbeat)

0    0.582035
4    0.198995
2    0.155402
1    0.055905
3    0.007663
Name: Target, dtype: float64

# Build a cross-sectional shallow model using Keras (with only one hidden layer) (2 points)

In [32]:
import tensorflow as tf
from tensorflow import keras

# fix random seed for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

In [33]:
#What is your input shape?
#(meaning: how many neurons should be in the input layer?)

train_x.shape

(5572, 80, 1)

In [35]:
#Define the model: for multi-class

model = keras.models.Sequential()

model.add(keras.layers.Input(shape=80))
model.add(keras.layers.Dense(50, activation='relu'))
model.add(keras.layers.Dense(5, activation='softmax'))

#final layer: there has to be 5 nodes with softmax (because we have 5 categories)

In [36]:
# Compile model

#Optimizer:
adam = keras.optimizers.Adam(learning_rate=0.01)

model.compile(loss='sparse_categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

In [37]:
# Fit the model

history = model.fit(train_x, train_y, 
                    validation_data=(test_x, test_y), 
                    epochs=20, batch_size=500)

Epoch 1/20
12/12 [==============================] - 1s 22ms/step - loss: 1.1176 - accuracy: 0.6179 - val_loss: 0.8627 - val_accuracy: 0.7098
Epoch 2/20
12/12 [==============================] - 0s 7ms/step - loss: 0.7950 - accuracy: 0.7554 - val_loss: 0.7314 - val_accuracy: 0.7580
Epoch 3/20
12/12 [==============================] - 0s 6ms/step - loss: 0.7184 - accuracy: 0.7758 - val_loss: 0.6673 - val_accuracy: 0.7919
Epoch 4/20
12/12 [==============================] - 0s 5ms/step - loss: 0.6671 - accuracy: 0.7902 - val_loss: 0.6335 - val_accuracy: 0.7973
Epoch 5/20
12/12 [==============================] - 0s 5ms/step - loss: 0.6353 - accuracy: 0.7895 - val_loss: 0.5866 - val_accuracy: 0.8199
Epoch 6/20
12/12 [==============================] - 0s 5ms/step - loss: 0.5934 - accuracy: 0.8074 - val_loss: 0.5651 - val_accuracy: 0.8216
Epoch 7/20
12/12 [==============================] - 0s 5ms/step - loss: 0.5586 - accuracy: 0.8175 - val_loss: 0.5332 - val_accuracy: 0.8162
Epoch 8/20
12/12 [=

In [38]:
# evaluate the model

scores = model.evaluate(test_x, test_y, verbose=0)

scores

# In results, first is loss, second is accuracy

[0.3726443350315094, 0.8840033411979675]

In [39]:
# extract the accuracy from model.evaluate

print("%s: %.2f" % (model.metrics_names[0], scores[0]))
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

loss: 0.37
accuracy: 88.40%


# Build a cross-sectional deep model using Keras (with two or more hidden layers) (2 points)

In [41]:
#Define the model: for multi-class

model = keras.models.Sequential()

model.add(keras.layers.Input(shape=80))
model.add(keras.layers.Dense(80, activation='relu'))
model.add(keras.layers.Dense(80, activation='relu'))
model.add(keras.layers.Dense(80, activation='relu'))
model.add(keras.layers.Dense(5, activation='softmax'))

#final layer: there has to be 5 nodes with softmax (because we have 5 categories)

In [42]:
# Compile model

#Optimizer:
adam = keras.optimizers.Adam(learning_rate=0.01)

model.compile(loss='sparse_categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

In [43]:
# Fit the model

history = model.fit(train_x, train_y, 
                    validation_data=(test_x, test_y), 
                    epochs=20, batch_size=500)

Epoch 1/20
12/12 [==============================] - 1s 31ms/step - loss: 1.1521 - accuracy: 0.5695 - val_loss: 0.8353 - val_accuracy: 0.7048
Epoch 2/20
12/12 [==============================] - 0s 7ms/step - loss: 0.6805 - accuracy: 0.7758 - val_loss: 0.5928 - val_accuracy: 0.8049
Epoch 3/20
12/12 [==============================] - 0s 7ms/step - loss: 0.5735 - accuracy: 0.8067 - val_loss: 0.5168 - val_accuracy: 0.8346
Epoch 4/20
12/12 [==============================] - 0s 7ms/step - loss: 0.5003 - accuracy: 0.8300 - val_loss: 0.4587 - val_accuracy: 0.8572
Epoch 5/20
12/12 [==============================] - 0s 8ms/step - loss: 0.4453 - accuracy: 0.8518 - val_loss: 0.3993 - val_accuracy: 0.8698
Epoch 6/20
12/12 [==============================] - 0s 7ms/step - loss: 0.3837 - accuracy: 0.8823 - val_loss: 0.4101 - val_accuracy: 0.8903
Epoch 7/20
12/12 [==============================] - 0s 6ms/step - loss: 0.3609 - accuracy: 0.8991 - val_loss: 0.3367 - val_accuracy: 0.9070
Epoch 8/20
12/12 [=

In [44]:
# evaluate the model

scores = model.evaluate(test_x, test_y, verbose=0)

scores

# In results, first is loss, second is accuracy

[0.23919467628002167, 0.9258794188499451]

In [45]:
# extract the accuracy from model.evaluate

print("%s: %.2f" % (model.metrics_names[0], scores[0]))
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

loss: 0.24
accuracy: 92.59%


# Build a sequential shallow LSTM Model (with only one LSTM layer) (2 points)

In [79]:
n_steps = 80
n_inputs = 1

model = keras.models.Sequential([
    
    keras.layers.LSTM(5, activation='sigmoid' , input_shape=[n_steps, n_inputs])
])

In [80]:
np.random.seed(42)
tf.random.set_seed(42)

optimizer = keras.optimizers.Nadam(learning_rate=0.01)

model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=['accuracy'])

history = model.fit(train_x, train_y, epochs=20,
                   validation_data = (test_x, test_y), callbacks=callback)

Epoch 1/20
175/175 [==============================] - 14s 64ms/step - loss: 1.2063 - accuracy: 0.5793 - val_loss: 1.1172 - val_accuracy: 0.5884
Epoch 2/20
175/175 [==============================] - 10s 59ms/step - loss: 1.1190 - accuracy: 0.5793 - val_loss: 1.0991 - val_accuracy: 0.5884
Epoch 3/20
175/175 [==============================] - 11s 62ms/step - loss: 1.1045 - accuracy: 0.5793 - val_loss: 1.0852 - val_accuracy: 0.5884
Epoch 4/20
175/175 [==============================] - 10s 59ms/step - loss: 1.0887 - accuracy: 0.5793 - val_loss: 1.0738 - val_accuracy: 0.5884
Epoch 5/20
175/175 [==============================] - 10s 57ms/step - loss: 1.0740 - accuracy: 0.5793 - val_loss: 1.0604 - val_accuracy: 0.5884
Epoch 6/20
175/175 [==============================] - 10s 58ms/step - loss: 1.0623 - accuracy: 0.5793 - val_loss: 1.0470 - val_accuracy: 0.5884
Epoch 7/20
175/175 [==============================] - 11s 60ms/step - loss: 1.0588 - accuracy: 0.5793 - val_loss: 1.0521 - val_accuracy:

In [81]:
# evaluate the model

scores = model.evaluate(test_x, test_y, verbose=0)

scores

# In results, first is loss, second is accuracy

[1.0263375043869019, 0.6298157572746277]

In [82]:
# extract the accuracy from model.evaluate

print("%s: %.2f" % (model.metrics_names[0], scores[0]))
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

loss: 1.03
accuracy: 62.98%


# Build a sequential deep LSTM Model (with only two LSTM layers) (2 points)

In [130]:
n_steps = 80
n_inputs = 1

model = keras.models.Sequential([
    keras.layers.LSTM(5,input_shape=[n_steps, n_inputs]),
    keras.layers.Dense(5, activation='softmax')
])

In [131]:
np.random.seed(42)
tf.random.set_seed(42)

optimizer = keras.optimizers.Nadam(learning_rate=0.01)
from tensorflow.keras.callbacks import EarlyStopping


earlystop = EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='auto')
callback = [earlystop]


model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=['accuracy'])

history = model.fit(train_x, train_y, epochs=20,
                   validation_data = (test_x, test_y), callbacks=callback)

Epoch 1/20
175/175 [==============================] - 8s 31ms/step - loss: 1.1513 - accuracy: 0.5723 - val_loss: 1.0592 - val_accuracy: 0.5884
Epoch 2/20
175/175 [==============================] - 5s 29ms/step - loss: 1.0626 - accuracy: 0.5739 - val_loss: 1.0331 - val_accuracy: 0.5884
Epoch 3/20
175/175 [==============================] - 5s 28ms/step - loss: 1.0112 - accuracy: 0.5996 - val_loss: 0.9521 - val_accuracy: 0.6382
Epoch 4/20
175/175 [==============================] - 5s 28ms/step - loss: 0.9208 - accuracy: 0.6457 - val_loss: 0.8964 - val_accuracy: 0.6499
Epoch 5/20
175/175 [==============================] - 5s 28ms/step - loss: 0.8054 - accuracy: 0.7094 - val_loss: 0.7825 - val_accuracy: 0.7341
Epoch 6/20
175/175 [==============================] - 5s 28ms/step - loss: 0.7171 - accuracy: 0.7674 - val_loss: 0.6434 - val_accuracy: 0.8078
Epoch 7/20
175/175 [==============================] - 5s 28ms/step - loss: 0.7851 - accuracy: 0.7414 - val_loss: 0.6984 - val_accuracy: 0.7864

In [92]:
# evaluate the model

scores = model.evaluate(test_x, test_y, verbose=0)

scores

# In results, first is loss, second is accuracy

[0.5719574093818665, 0.8245393633842468]

In [93]:
# extract the accuracy from model.evaluate

print("%s: %.2f" % (model.metrics_names[0], scores[0]))
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

loss: 0.57
accuracy: 82.45%


# Build a sequential shallow GRU Model (with only one GRU layer) (2 points)

In [124]:
n_steps = 80
n_inputs = 1

model = keras.models.Sequential([
    keras.layers.GRU(5, input_shape=[n_steps, n_inputs]),
    tf.keras.layers.Dense(5, activation='softmax')
])

In [125]:
np.random.seed(42)
tf.random.set_seed(42)

optimizer = keras.optimizers.Nadam(learning_rate=0.01)
from tensorflow.keras.callbacks import EarlyStopping


earlystop = EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='auto')
callback = [earlystop]


model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=['accuracy'])

history = model.fit(train_x, train_y, epochs=20,
                   validation_data = (test_x, test_y), callbacks=callback)

Epoch 1/20
175/175 [==============================] - 9s 37ms/step - loss: 1.1369 - accuracy: 0.5750 - val_loss: 1.0408 - val_accuracy: 0.6181
Epoch 2/20
175/175 [==============================] - 6s 34ms/step - loss: 1.0496 - accuracy: 0.6052 - val_loss: 1.0342 - val_accuracy: 0.5925
Epoch 3/20
175/175 [==============================] - 6s 33ms/step - loss: 1.0347 - accuracy: 0.6043 - val_loss: 1.0735 - val_accuracy: 0.5745
Epoch 4/20
175/175 [==============================] - 6s 34ms/step - loss: 1.0245 - accuracy: 0.5976 - val_loss: 1.0134 - val_accuracy: 0.5925
Epoch 5/20
175/175 [==============================] - 6s 34ms/step - loss: 1.0183 - accuracy: 0.5998 - val_loss: 0.9983 - val_accuracy: 0.6164
Epoch 6/20
175/175 [==============================] - 6s 34ms/step - loss: 1.0081 - accuracy: 0.6039 - val_loss: 0.9972 - val_accuracy: 0.6223
Epoch 7/20
175/175 [==============================] - 6s 34ms/step - loss: 0.9972 - accuracy: 0.6192 - val_loss: 0.9954 - val_accuracy: 0.6512

In [126]:
# evaluate the model

scores = model.evaluate(test_x, test_y, verbose=0)

scores

# In results, first is loss, second is accuracy

[0.5190359950065613, 0.8262144327163696]

In [127]:
# extract the accuracy from model.evaluate

print("%s: %.2f" % (model.metrics_names[0], scores[0]))
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

loss: 0.52
accuracy: 82.62%


# Build a sequential deep GRU Model (with only two GRU layers) (2 points)

In [120]:
n_steps = 80
n_inputs = 1

model = keras.models.Sequential([
    keras.layers.GRU(2, return_sequences=True, input_shape=[n_steps, n_inputs]),
    keras.layers.GRU(2, return_sequences=True),
    keras.layers.GRU(2),
    keras.layers.Dense(1, activation='softmax')
])




In [121]:
np.random.seed(42)
tf.random.set_seed(42)

optimizer = keras.optimizers.Nadam(learning_rate=0.15)

model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=['accuracy'])

from tensorflow.keras.callbacks import EarlyStopping


earlystop = EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='auto')
callback = [earlystop]

history = model.fit(train_x, train_y, epochs=20,
                   validation_data = (test_x, test_y), callbacks=callback)


Epoch 1/20
175/175 [==============================] - 25s 101ms/step - loss: -4.4257 - accuracy: 0.0564 - val_loss: -7.8710 - val_accuracy: 0.0549
Epoch 2/20
175/175 [==============================] - 17s 98ms/step - loss: -13.0934 - accuracy: 0.0564 - val_loss: -15.5882 - val_accuracy: 0.0549
Epoch 3/20
175/175 [==============================] - 18s 102ms/step - loss: -21.7605 - accuracy: 0.0564 - val_loss: -23.1215 - val_accuracy: 0.0549
Epoch 4/20
175/175 [==============================] - 17s 100ms/step - loss: -30.2244 - accuracy: 0.0564 - val_loss: -30.6095 - val_accuracy: 0.0549
Epoch 5/20
175/175 [==============================] - 17s 100ms/step - loss: -38.7439 - accuracy: 0.0564 - val_loss: -38.2967 - val_accuracy: 0.0549
Epoch 6/20
175/175 [==============================] - 17s 100ms/step - loss: -47.4165 - accuracy: 0.0564 - val_loss: -45.7897 - val_accuracy: 0.0549
Epoch 7/20
175/175 [==============================] - 18s 101ms/step - loss: -55.7867 - accuracy: 0.0564 - va

In [122]:
# evaluate the model

scores = model.evaluate(test_x, test_y, verbose=0)

scores

# In results, first is loss, second is accuracy

[-151.46958923339844, 0.05485762283205986]

In [123]:
# extract the accuracy from model.evaluate

print("%s: %.2f" % (model.metrics_names[0], scores[0]))
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

loss: -151.47
accuracy: 5.49%


# Discussion

## List the test values of each model you built (0.5 points)

## Which model performs the best and why? (0.5 points) 
## How does it compare to baseline? (0.5 points)

